In [5]:
# Loading the aiida profile.
from aiida import load_profile, orm
load_profile()

# Loading other additional aiida modules.
from aiida.engine import submit, run_get_node
from aiida.plugins import DataFactory

# Loading the pymatgen Structure module; not mandatory.
from pymatgen.core import Structure

# Loading the WorkChain.
from aiida_muon.workflows.find_muon import FindMuonWorkChain

In [6]:
# StructureData and magmom.
smag1 = Structure.from_file("/home/jovyan/bind_mount/codes/aiida-muon/examples/data/MnO.mcif", primitive=False)
aiida_structure = orm.StructureData(pymatgen=smag1)

magmoms = smag1.site_properties["magmom"]
magmom = [list(magmom) for magmom in magmoms]

# Supercell Matrix and other relevant inputs.
sc_matrix = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
mu_spacing = 1.0
kpoints_distance = 2.0
charge_supercell = True

# Codes.
codename = "pw-7.3@thor_1-6_final"  # edit 
code = orm.load_code(codename)
pp_codename = "pp-7.3@thor_1-6_final"  # edit 
pp_code = orm.load_code(pp_codename)

# Resources dictionary with a minimal computational settings.
resources = {
    "num_machines": 1,
    "num_mpiprocs_per_machine": 48,
}

/opt/conda/lib/python3.9/site-packages/pymatgen/core/structure.py:3083: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:


In [9]:
builder = FindMuonWorkChain.get_builder_from_protocol(
    pw_code=code,
    pp_code = pp_code,
    structure = aiida_structure, # orm.load_node(21259), # <- LiF
    magmom = magmom,
    hubbard=True,
    hubbard_dict = {"Mn":5},
    sc_matrix = sc_matrix,
    mu_spacing = mu_spacing,
    kpoints_distance = kpoints_distance,
    charge_supercell = charge_supercell,
    pseudo_family = "SSSP/1.3/PBE/efficiency",
    enforce_defaults = True, # if True, it will set some default parameter.
    )

builder.pwscf.pw.metadata.options.resources = resources
builder.pwscf.pw.metadata.options.prepend_text = "export OMP_NUM_THREADS=1"

builder.relax.base.pw.metadata.options.resources = resources
builder.relax.base.pw.metadata.options.prepend_text = "export OMP_NUM_THREADS=1"

builder.impuritysupercellconv_metadata = {"options":{
    'resources':resources,
    'prepend_text':"export OMP_NUM_THREADS=1",
    },}

builder.pp_metadata = {"options":{
    'resources':resources,
    'prepend_text':"export OMP_NUM_THREADS=1",
    'max_wallclock_seconds': 1*60*60, # 1h
    },}

{'Mn1': 5, 'Mn2': 5} {'Mn': 5}
